# Question 1

In [ ]:
# 2)
import numpy as np
# a)
sample = np.array([4, 7, 9, 11, 15])
var = np.var(sample, ddof=1)  # use ddof of 1 because sample
print(f'Variance for a): {var}')

Variance for a): 17.2


In [ ]:
# b)
sample = np.array([18, 22, 25, 30, 35, 40])
var = np.var(sample, ddof=1)  # use ddof of 1 because sample
print(f'Variance for b): {var}')

Variance for b): 68.26666666666668


In [7]:
# 4)

# a)
sample = np.array([12, 14, 16, 18, 20])
var = np.var(sample, ddof=0)  # use ddof 0 for population
print(f'Variance for a): {var}')

Variance for a): 8.0


In [8]:
# b)
sample = np.array([28, 32, 36, 40, 44, 48])
var = np.var(sample, ddof=0)  # use ddof 0 for population
print(f'Variance for b): {var}')

Variance for b): 46.666666666666664


5)
a): They key difference between empirical and population variance calculations is when calculation the empirical variance, the denominator used is n-1 while for the population variance the denominator used is n <br>
b): An example of where you would want to use population variance over empirical variance is for example if you wanted to find the variance of the ages of kids in a specific class. However, if you wanted to find the variance in ages for an entire school, it would be better to take a sample of kids' ages and then use the empirical variance in your calculation.

# Question 2

In [40]:
import numpy as np
lambda_ = 2
eta = 0.5

X = np.array([[1, -2, 0, 1], [-2, -1, 1, 2], [1, 2, -1, 1]])
w0 = np.array([[-1], [0], [1], [1]])
Y = np.array([[2], [3], [-1]])

def f_prime(w):
    return 2 * (X.T.dot(X.dot(w)-Y) + 2 * w)

w = w0
for i in range(2):
    w = w - eta * f_prime(w)

print(f'W after two steps of gradient descent: {w}')

W after two steps of gradient descent: [[-65.]
 [-18.]
 [ 31.]
 [ 41.]]


# Question 3

In [3]:
import numpy as np
from numpy import genfromtxt

X = genfromtxt('stock_prediction_data.csv', delimiter=',')

y = genfromtxt('stock_price.csv', delimiter=',')
y = np.reshape(y, (len(y),1))

In [4]:
# Split data
from sklearn.model_selection import train_test_split

X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_rest, y_rest, test_size=0.5)